# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)

## Background

### Target details
Epidermal Growth Factor Receptor (EGFR) is a transmembrane protein that plays a critical role in cell growth, differentiation, and survival. It is frequently overexpressed or mutated in various cancers, including non-small cell lung cancer, colorectal cancer, and head and neck cancer. This makes EGFR a crucial target for cancer therapies such as Cetuximab, an antibody with more than 1B USD in annual revenue. 

- Target Protein: EGFR
- Organism: HUMAN
- Uniprot Accession ID: [P00533](https://www.uniprot.org/uniprotkb/P00533/entry)
- Protein sequence: LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPS
- Structure PDB: [6ARU](https://www.rcsb.org/structure/6aru)


![64ru](https://cdn.rcsb.org/images/structures/6aru_assembly-1.jpeg)

### Binding protein designs
This dataset contains 202 designed EGFR-binding protein sequences, along with experimental binding affinity results tested by the AdaptyvBio team.

## Reference:
- https://design.adaptyvbio.com/
- https://foundry.adaptyvbio.com/egfr_design_competition

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation
from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))

In [3]:
# Get the owner and organization
org = "AdaptyvBio"
data_name = "EGFR_binders"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug="adaptyv-bio", type="organization")
owner

HubOwner(slug='adaptyv-bio', external_id=None, type='organization')

In [4]:
BENCHMARK_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/benchmarks"
DATASET_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/datasets"
FIGURE_DIR = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/figures"

## Load existing data

In [5]:
# Load the curated data
PATH = "gs://polaris-public/polaris-recipes/org-AdaptyvBio/EGFR_binders/raw/Competition_Binders_to_EGFR_mean.csv"
table = pd.read_csv(PATH)

### Below we specify the meta information of data columns

It's necessary to specify the key bioactivity columns, molecule structures and identifiers in the dataset with `ColumnAnnotation`. It is possible to add `user_attributes` with any key and values when needed, such as `unit`, `organism`, `scale` and optimization `objective`. 

This dataset includes two weak binders. Since only six designs exhibit moderate to strong binding affinities, these two weak binders are classified as positive binders in the binary classification setting. The `unknown` binding results are labeled as `False`.


In [6]:
# fill nans
table["binding"] = table["binding"].replace("weak", "True").values
table["binding"] = table["binding"].replace("unknown", "False").values
table["binding"] = table["binding"] == "True"  # to boolean
table["binding"].value_counts()

binding
False    194
True       8
Name: count, dtype: int64

In [7]:
# Rename column names for compheransive terms
table.rename(columns={"kd": "KD", "binding": "binding_class"}, inplace=True)

In [8]:
annotations = {
    "name": ColumnAnnotation(description="Sequence design name."),
    "sequence": ColumnAnnotation(description="Protein sequence in fasta format."),
    "KD": ColumnAnnotation(
        description="The equilibrium dissociation constant (KD) for the measure of binding affinity.",
        user_attributes={
            "unit": "M",
            "objective": "Lower value",
        },
    ),
    "binding_class": ColumnAnnotation(
        description="The binding affinity as boolean classes labels.",
        user_attributes={
            "objective": "True",
        },
    ),
}

### Define `Dataset` object

In [9]:
dataset_version = "v1"
dataset_name = f"{data_name}-{dataset_version}"

In [10]:
dataset = Dataset(
    table=table[annotations.keys()].copy(),
    name=dataset_name,
    description="This dataset includes binding protein designs targeting the Epidermal growth factor receptor(EGFR), a drug target associated with various diseases.",
    source="https://design.adaptyvbio.com/",
    annotations=annotations,
    owner=owner,
    tags=["protein-design"],
    license="CC-BY-4.0",
)

### Dataset overview

In [11]:
dataset

{
  "name": "EGFR_binders-v1",
  "description": "This dataset includes binding protein designs targeting the Epidermal growth factor receptor(EGFR), a drug target associated with various diseases.",
  "tags": [
    "protein-design"
  ],
  "user_attributes": {},
  "owner": "adaptyv-bio",
  "polaris_version": "0.8.5.dev6+gb44821c",
  "default_adapters": {},
  "zarr_root_path": null,
  "readme": "",
  "annotations": {
    "name": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "Sequence design name.",
      "user_attributes": {},
      "dtype": "object",
      "content_type": null
    },
    "sequence": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "Protein sequence in fasta format.",
      "user_attributes": {},
      "dtype": "object",
      "content_type": null
    },
    "KD": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "The equilibrium dissociation constant (KD) for the measure of binding affinity.",
      "user_attributes": {
        "unit": "M",
        "objective": "Lower value"
      },
      "dtype": "float64",
      "content_type": null
    },
    "binding_class": {
      "is_pointer": false,
      "modality": "UNKNOWN",
      "description": "The binding affinity as boolean classes labels.",
      "user_attributes": {
        "objective": "True"
      },
      "dtype": "bool",
      "content_type": null
    }
  },
  "source": "https://design.adaptyvbio.com/",
  "license": "CC-BY-4.0",
  "curation_reference": null,
  "cache_dir": "/Users/lu.zhu/Library/Caches/polaris/datasets/3fe165a8-3f84-4a4e-a107-c1db7ee73c8b",
  "md5sum": "e7c851d24c4d8062871f1050f742dc73",
  "artifact_id": "adaptyv-bio/egfr-binders-v1",
  "n_rows": 202,
  "n_columns": 4
}

In [12]:
# # save the dataset to GCP
# SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
# dataset_path = dataset.to_json(SAVE_DIR)
# dataset_path

### Upload the dataset to the hub

In [13]:
dataset.upload_to_hub()

✅ SUCCESS: Your standard dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/adaptyv-bio/EGFR_binders-v1
 


/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


In [15]:
# test to load the benchmark

import polaris as po

# Load the benchmark from the Hub
benchmark = po.load_benchmark("adaptyv-bio/EGFR_binders_binary_cls-v1")

# Get the train and test data-loaders
train, test = benchmark.get_train_test_split()

⠦ Fetching artifact... 

2024-09-24 11:22:09.695 | INFO     | polaris.mixins._checksum:verify_checksum:65 - To verify the checksum, we need to recompute it. This can be slow for large datasets.
/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
2024-09-24 11:22:09.701 | INFO     | polaris.benchmark._base:_validate_split:189 - This benchmark only specifies a test set. It will return an empty train set in `get_train_test_split()`
2024-09-24 11:22:09.702 | WARNING  | polaris.mixins._checksum:verify_checksum:58 - No checksum to verify against. Specify either the md5sum parameter or store the checksum in the dataset.md5sum attribute.


✅ SUCCESS: Fetched artifact.
 
✅ SUCCESS: Fetched artifact.
 


In [16]:
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    explained_variance_score,
    f1_score,
    matthews_corrcoef,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    roc_auc_score,
    balanced_accuracy_score,
)

In [17]:
benchmark.dataset.table["binding_class"].unique()

array([ True, False])

In [18]:
balanced_accuracy_score(predictions, predictions)

NameError: name 'predictions' is not defined

In [19]:
import numpy as np

# Work your magic to accurately predict the test set
predictions = np.array([True for x in test])

# Evaluate your predictions
results = benchmark.evaluate(predictions)

# Submit your results
# results.upload_to_hub(owner="lu-valencelabs")

In [20]:
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.8.5.dev6+gb44821c",
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "benchmark_name": "EGFR_binders_binary_cls-v1",
  "benchmark_owner": {
    "slug": "adaptyv-bio",
    "external_id": "org_2lqe2oSyR0fGZEEVd1NUlwqnfjx",
    "type": "organization"
  },
  "artifact_id": null,
  "benchmark_artifact_id": "adaptyv-bio/egfr-binders-binary-cls-v1",
  "results": [
    {
      "Test set": "test",
      "Target label": "binding_class",
      "Metric": "accuracy",
      "Score": 0.0396039604
    },
    {
      "Test set": "test",
      "Target label": "binding_class",
      "Metric": "balanced_accuracy",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "binding_class",
      "Metric": "f1",
      "Score": 0.0761904762
    },
    {
      "Test set": "test",
      "Target label": "binding_class",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "binding_class",
      "Metric": "cohen_kappa",
      "Score": 0.0
    }
  ]
}